In [59]:
import networkx as nx
import numpy as np
import scipy.sparse as sp
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from tqdm.notebook import tqdm


In [63]:
# Simulate N graphs
NUM_GRAPHS = 101
MIN_NUM_NODES, MAX_NUM_NODES = 50, 100
MIN_EDGE_PER_NODE, MAX_EDGE_PER_NODE = 2, 5
hidden_dim = 32  # dimensionality of SVD
top_percentile = 90  # centrality percentile over which the node label is equal to 1

x_train = None
y_train = None
for i in tqdm(range(NUM_GRAPHS-1)):
    # Step 1. Generate the graph
    num_nodes = np.random.randint(low=MIN_NUM_NODES, high=MAX_NUM_NODES)
    num_edges_per_node = np.random.randint(low=MIN_EDGE_PER_NODE, high=MAX_EDGE_PER_NODE)
    graph = nx.barabasi_albert_graph(n=num_nodes,
                                     m=num_edges_per_node, seed=i)
    # Step 2. Compute Singular Value Decomposition of the adjacency matrix
    adj_matrix = nx.to_numpy_array(graph)
    row, col = np.where(adj_matrix == 1.)
    sparse_adj_matrix = sp.coo_matrix((np.ones(row.shape[0]), (row, col)), shape=(num_nodes, num_nodes))
    svd = TruncatedSVD(n_components=hidden_dim, n_iter=128)
    svd.fit(sparse_adj_matrix)
    svd_embeddings = svd.components_.T
    # Step 3. Label each node according to its eigenvector centrality
    eigenvector_centrality_dict = nx.eigenvector_centrality(graph)
    eigenvector_centrality_list = [-1]*graph.number_of_nodes()
    for node_id in eigenvector_centrality_dict:
        eigenvector_centrality_list[node_id] = eigenvector_centrality_dict[node_id]
    eigenvector_centrality_list = np.array(eigenvector_centrality_list)
    
    top_percentile_value = np.percentile(eigenvector_centrality_list, top_percentile)
    y_graph_label = (eigenvector_centrality_list > top_percentile_value).astype(int)
    if i == 0:
        x_train = np.copy(svd_embeddings)
        y_train = np.copy(y_graph_label)
    else:
        x_train = np.vstack([x_train, np.copy(svd_embeddings)])
        y_train = np.concatenate([y_train, np.copy(y_graph_label)])


  0%|          | 0/100 [00:00<?, ?it/s]

In [75]:
# Step 1. Generate a test graph
num_nodes = np.random.randint(low=MIN_NUM_NODES, high=MAX_NUM_NODES)
num_edges_per_node = np.random.randint(low=MIN_EDGE_PER_NODE, high=MAX_EDGE_PER_NODE)
test_graph = nx.barabasi_albert_graph(n=num_nodes,
                                      m=num_edges_per_node, seed=NUM_GRAPHS)
# Step 2. Compute Singular Value Decomposition of the adjacency matrix
adj_matrix = nx.to_numpy_array(test_graph)
row, col = np.where(adj_matrix == 1.)
sparse_adj_matrix = sp.coo_matrix((np.ones(row.shape[0]), (row, col)), shape=(num_nodes, num_nodes))
svd = TruncatedSVD(n_components=hidden_dim, n_iter=128)
svd.fit(sparse_adj_matrix)
test_svd_embeddings = svd.components_.T
# Step 3. Label each node according to its eigenvector centrality
eigenvector_centrality_dict = nx.eigenvector_centrality(test_graph)
eigenvector_centrality_list = [-1]*test_graph.number_of_nodes()
for node_id in eigenvector_centrality_dict:
    eigenvector_centrality_list[node_id] = eigenvector_centrality_dict[node_id]
eigenvector_centrality_list = np.array(eigenvector_centrality_list)
top_percentile_value = np.percentile(eigenvector_centrality_list, top_percentile)
y_test_graph_label = (eigenvector_centrality_list > top_percentile_value).astype(int)


In [76]:
# Step 1. Train a model on training graphs
classification_model = LogisticRegression()
classification_model.fit(x_train, y_train)

# Step 2. Make predictions on test graph
y_logits = classification_model.predict_proba(test_svd_embeddings)[:, 1]
y_pred = classification_model.predict(test_svd_embeddings)

# Step 3. Evaluate predictions
test_metrics_dict = {'f1_macro': metrics.f1_score(y_test_graph_label,
                                                  y_pred, average='macro'),
                     'f1_micro': metrics.f1_score(y_test_graph_label,
                                                  y_pred, average='micro'),
                     'accuracy': metrics.accuracy_score(y_test_graph_label,
                                                        y_pred),
                     'precision': metrics.precision_score(y_test_graph_label,
                                                          y_pred),
                     'roc_auc': metrics.roc_auc_score(y_test_graph_label,
                                                          y_logits)}

print('Result on test set:')
print(test_metrics_dict)


Result on test set:
{'f1_macro': 0.791969037252056, 'f1_micro': 0.9418604651162791, 'accuracy': 0.9418604651162791, 'precision': 1.0, 'roc_auc': 1.0}
